In [2]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *


import os,sys
from importlib import reload
import copy
sys.path.append("../tracker")

In [4]:
import KalmanFilter as KF
import KalmanUtils as KU
import TrackFinder as TF
import kf_debug as debug
import DataTypes

reload(KF)
reload(KU)
reload(debug)

<module 'kf_debug' from '../tracker\\kf_debug.py'>

In [5]:
# Generated toy Events

N_events=500
Tracks_per_event = 4
N_tracks = N_events * Tracks_per_event
Hit_collection={}
Truth_collection ={}
ParTruth_collection={}

x0s = np.random.uniform(-100,100,N_tracks)
z0s = np.random.uniform(-100,100,N_tracks)
t0s = np.random.uniform(-3,3,N_tracks)
Axs = np.random.uniform(-0.3,0.3,N_tracks)
Azs = np.random.uniform(-0.3,0.3,N_tracks)
Ats = np.random.uniform(1/28,1/32,N_tracks)

for i in range(N_events):
    # Make multiple tracks per event    
    tracks, truths, partruth = [],[],[]
    for j in range(Tracks_per_event):
        k =Tracks_per_event*i+j
        a,b,c = debug.gen_hits(x0=x0s[k],y0=0,z0=z0s[k], t0=t0s[k], Ax=Axs[k],Az=Azs[k],At=Ats[k],  N_LAYERS=4)
        tracks.extend(a)
        truths.extend(b)
        partruth.extend(c)

    Hit_collection[i],Truth_collection[i],ParTruth_collection[i] = tracks, truths, partruth




### Confirm that chi2_predict equals to chi2_filtered

In [8]:
hits = Hit_collection[4][:4]
n_hits = len(hits)

kf_find = KF.KalmanFilterFind()

# Set initial state using first two hits
m0, V0, H0, Xf0, Cf0, Rf0 = KU.init_state(hits)
kf_find.init_filter( m0, V0, H0, Xf0, Cf0, Rf0)


# Feed all measurements to KF
for i in range(2,n_hits):   
    # some processing
    hit = hits[i]
    dy  = hits[i].y-hits[i-1].y
    mi, Vi, Hi, Fi, Qi = KU.add_measurement(hit, dy)
    
    # pass to KF
    kf_find.update_matrix(Vi, Hi, Fi, Qi)
    chi2_predict = kf_find.forward_predict_chi2(mi)
    chi2_filtered = kf_find.forward_filter(mi)
    print("chi2 chi2_predict, chi2_filtered", chi2_predict, chi2_filtered)


kf = KU.run_kf(hits)
print(kf.chift)

print(kf.Xf[-1])
print(kf_find.Xf)


chi2 chi2_predict, chi2_filtered 2.443934787768825 2.4439347877688666
chi2 chi2_predict, chi2_filtered 6.647430711513302 6.6474307115139295
[0, 2.4439347877688666, 6.6474307115139295]
[-6.77050063e+01 -2.48974361e+00  6.95822890e+00  2.44423237e-02
 -8.53247289e-02  2.93005720e-02]
[-6.77050063e+01 -2.48974361e+00  6.95822890e+00  2.44423237e-02
 -8.53247289e-02  2.93005720e-02]


## Track finder algorithm Outline


Pseudo code:

```python
hits # provide a list of hits
seeds = seed(hts) # Make a list of seed
while len(seeds)>0:
    seed = seeds[0]

    # Round 1: find hits
    hits_found = find()
    seed.pop(0) # Remove the used seed
    continue if len(hits_found)<nhits_min

    # Round 2: run KF and drop outlier hits during smoothing
    hits_found = run_kf_drop()
    continue if len(hits_found)<nhits_min

    # Round 3: run KF (no dropping)
    run_kf(drop=False)
    remove_related_hits_seeds()

```

In [17]:
import numpy as np
from numpy.linalg import inv
import scipy as sp
import scipy.constants


import KalmanUtils as KU
import KalmanFilter as KF

import copy

# ----------------------------------------------------------------------
class TrackFinder:
    def __init__(self, parameters=None, method="recursive"):
        self.method = method # ("recursive", "greedy")
        self.parameters={
            "cut_track_HitAddChi2": 10,
            "cut_track_HitDropChi2": 10, # Set to -1 to turn off
            "cut_track_HitProjectionSigma": 10, # Number of sigmas
            "cut_track_TrackChi2Reduced": 5,
            "cut_track_TrackNHitsMin": 3,
        }

    def run(self, hits):
        """
        Run all three rounds of filtering: find, drop, fit
        This function is a mixer of self.find() and self.filter_smooth()
        """
        self.seeds = self.seeding(hits)
        self.hits = copy.copy(hits)
        self.hits_grouped = KU.group_hits_by_layer(self.hits)
        self.tracks = []
        while len(self.seeds)>0:

            # ------------------------------------
            # Round 1: Find hits that belongs to one track
            seed = self.seeds[0]
            hits_found, track_chi2 = self.find_once(self.hits, self.hits_grouped, seed)
            # Apply cuts
            ndof = 3*len(hits_found) - 6
            track_chi2_reduced = track_chi2/ndof
            # If chi2 is too large, drop this track
            if track_chi2_reduced>self.parameters["cut_track_TrackChi2Reduced"]:
                continue
            # If not enough hits, drop this track
            if len(hits_found)<self.parameters["cut_track_TrackNHitsMin"]:
                continue

            # Sort the hits by time before running the filter
            hits_found.sort(key=lambda hit: hit.t)

            # ------------------------------------
            # Round 2: Run filter and smooth with the option to drop outlier during smoothing
            kalman_result, inds_dropped = self.filter_smooth(hits_found, drop_chi2=self.parameters["cut_track_HitDropChi2"])
            inds_dropped.sort(reverse=True)
            for ind in inds_dropped:
                hits_found.pop(ind)
            # If not enough hits, drop this track
            if len(hits_found)<self.parameters["cut_track_TrackNHitsMin"]:
                continue                

            # ------------------------------------
            # Round 3: Run filter and smooth again without dropping
            kalman_result, inds_dropped = self.filter_smooth(hits_found, drop_chi2=-1)   
            # Remove the seed that shares hits of the found track
            self.remove_related_hits_seeds(hits_found)

            # ------------------------------------
            # Finally, prepare the output
            track_output = self.prepare_output(kalman_result, hits_found, track_ind = len(self.tracks))  
            self.tracks.append(track_output)            



        return self.tracks

    def prepare_output(self, kalman_result, hits_found, track_ind=0):
        """ 
        Turn the Kalman filter result into a Track object

        """
        # propagate the KF result from the second hit to the first hit
        mi, Vi, Hi, Fi, Qi = KU.add_measurement(hits_found[0], hits_found[0].y - hits_found[1].y)
        state_predicted_step_0 = Fi@kalman_result.Xsm[0]
        statecov_predicted_step_0 = Fi@kalman_result.Csm[0]@Fi.T + Qi 
        # Add the covariance of one additional layer:
        cov = statecov_predicted_step_0 + Qi
        chi2 = kalman_result.chift_total
        ind = track_ind
        hits = [hit.ind for hit in hits_found]

        
        x0 = state_predicted_step_0[0]
        z0 = state_predicted_step_0[1]
        t0 = state_predicted_step_0[2]
        Ax = state_predicted_step_0[3]
        Az = state_predicted_step_0[4]
        At = state_predicted_step_0[5]

        y0 = hits_found[0].y
        Ay = 1 # Slope of Y vs Y, which is always 1

        # Track is a namedtuple("Track", ["x0", "y0", "z0", "t", "Ax", "Ay", "Az", "At", "cov", "chi2", "ind", "hits"])
        track = Track(x0, y0, z0, t0, Ax, Ay, Az, At, cov, chi2, ind, hits)
        return track

    def find(self, hits):
        self.seeds = self.seeding(hits)
        self.hits = copy.copy(hits)
        self.hits_grouped = KU.group_hits_by_layer(self.hits)
        self.tracks_found = []
        while len(self.seeds)>0:
            seed = self.seeds[0]
            hits_found, track_chi2 = self.find_once(self.hits, self.hits_grouped, seed)

            # Apply cuts
            ndof = 3*len(hits_found) - 6
            track_chi2_reduced = track_chi2/ndof
            # If chi2 is too large, drop this track
            if track_chi2_reduced>self.parameters["cut_track_TrackChi2Reduced"]:
                continue
            # If not enough hits, drop this track
            if len(hits_found)<self.parameters["cut_track_TrackNHitsMin"]:
                continue

            # Attach the track if it pass the cuts
            self.tracks_found.append(hits_found)

            # Remove the seed that are in the track
            self.remove_related_hits_seeds(hits_found)

        return self.tracks_found


    def seeding(self, hits):
        """
        Find seed for tracks
        Returns a pair of index of the hit (not the hit itself!) and a score
        """
        c=sp.constants.c/1e7 # [cm/ns]
        seeds=[]
        for i in range(len(hits)):
            for j in range(i+1, len(hits)):
                if hits[i].y == hits[j].y:
                    continue
                dx = hits[i].x- hits[j].x
                dy = hits[i].y- hits[j].y
                dz = hits[i].z- hits[j].z
                dt = hits[i].t- hits[j].t
                ds = np.abs((dx**2+dy**2-dz**2)/c**2-dt**2)
                if ds>50:
                    continue
                seeds.append([i,j,ds])
        return seeds

    def find_once(self, hits, hits_layer_grouped, seed):  
        #### General info ####
        LAYERS = np.sort(list(hits_layer_grouped.keys()))

        ##### Seed ####
        # Check the direction of seed by comparing the time of two hits
        seed_hits = [hits[seed[0]], hits[seed[1]]]
        # Alwayse have the first hit to be first in time
        if (seed_hits[0].t > seed_hits[1].t):
            seed_hits = seed_hits[::-1]
        seed_start_layer = seed_hits[0].layer
        seed_stop_layer  = seed_hits[1].layer        
        # Check if needed to find backward or forward
        if (seed_hits[0].y > seed_hits[1].y):
            TRACK_DIRECTION = 0 # Downward track
            FIND_FORWARD = seed_start_layer>=LAYERS[0]
            FIND_FORWARD_LAYERS = LAYERS[:np.argmax(LAYERS>seed_stop_layer)-1][::-1]
            FIND_BACKWARD = seed_stop_layer<LAYERS[-1]
            FIND_BACKWARD_LAYERS = LAYERS[np.argmax(LAYERS>seed_stop_layer):]     
        else:
            TRACK_DIRECTION = 1 # Upward track
            FIND_FORWARD = seed_stop_layer<LAYERS[-1]
            FIND_FORWARD_LAYERS  = LAYERS[np.argmax(LAYERS>seed_stop_layer):]
            FIND_BACKWARD = seed_start_layer>=LAYERS[0]
            FIND_BACKWARD_LAYERS = LAYERS[:np.argmax(LAYERS>seed_stop_layer)-1][::-1]


        ##### Find ####
        hits_found = [seed_hits[1]]
        chi2_found = 0
        # Find forward
        if FIND_FORWARD:
            step_pre = seed_hits[1].y # Keep track of the y of the previous step

            kf_find = KF.KalmanFilterFind()
            kf_find.init_filter(*KU.init_state(seed_hits)) # Set initial state using two hits specified by the seed
            if self.method=="recursive":
                hits_found_forward,chi2 = self.find_in_layers_recursive(hits, hits_layer_grouped, FIND_FORWARD_LAYERS, kf_find, step_pre)
            else:
                hits_found_forward,chi2 = self.find_in_layers_greedy(hits, hits_layer_grouped, FIND_FORWARD_LAYERS, kf_find, step_pre)
            chi2_found+=chi2
            hits_found.extend(hits_found_forward)

        if FIND_BACKWARD:
            step_pre = seed_hits[1].y # Keep track of the y of the previous step

            kf_find = KF.KalmanFilterFind()
            kf_find.init_filter(*KU.init_state(seed_hits))
            if self.method=="recursive":
                hits_found_backward, chi2 = self.find_in_layers_recursive(hits, hits_layer_grouped, FIND_BACKWARD_LAYERS, kf_find, step_pre)
            else:
                hits_found_backward, chi2 = self.find_in_layers_greedy(hits, hits_layer_grouped, FIND_BACKWARD_LAYERS, kf_find, step_pre)
            # Order of found hits also needs to be reversed for backward finding
            chi2_found+=chi2
            hits_found_backward = hits_found_backward[::-1] 
            hits_found_backward.extend(hits_found)   
            hits_found = hits_found_backward      

        return hits_found,chi2_found

    def find_in_layers_greedy(self, hits, hits_layer_grouped, layers_to_scan, kf_find, step_pre):
        """
        Find the hit that has minimum chi2 in each layer
        """
        hits_found = []
        for layer in layers_to_scan:
            hits_thislayer = hits_layer_grouped[layer]

            # Get the prediction matrix 
            step_this = hits_thislayer[0].y 
            dy = step_this - step_pre # Step size
            _, Vi, Hi, Fi, Qi = KU.add_measurement(hits_thislayer[0], dy) # Calculate matrices. Only need to do once for all this in the same layer
            kf_find.update_matrix(Vi, Hi, Fi, Qi) # pass matrices to KF

            # Use the Predicted location to limit the search range
            Xp = kf_find.Xp_i
            Xp_unc = np.sqrt(np.diag(kf_find.Rp_i))
            # Function to test if new measurement is within N_sigma times the uncertainty ellipsoid
            N_sigma = self.parameters["cut_track_HitProjectionSigma"]
            test_measurement_compatible = lambda x,z,t: ((x-Xp[0])/Xp_unc[0])**2 + ((z-Xp[1])/Xp_unc[1])**2 + ((t-Xp[2])/Xp_unc[2])**2 <N_sigma            

            # Calculate chi2 for all hits in the next layer
            # chi2_predict = [kf_find.forward_predict_chi2(np.array([mi.x, mi.z, mi.t])) for mi in hits_thislayer]
            chi2_predict=[]
            chi2_predict_inds =[]
            for imeasurement, m in enumerate(hits_thislayer):
                if not test_measurement_compatible(m.x, m.z, m.t):
                    continue
                else:
                    chi2 = kf_find.forward_predict_chi2(np.array([m.x, m.z, m.t]))
                    chi2_predict.append(chi2)
                    chi2_predict_inds.append(imeasurement)
            if len(chi2_predict)==0:
                return [],0


            # Find the hit with minimum chi2
            chi2_min_idx = np.argmin(chi2_predict)
            if chi2_predict[chi2_min_idx]<self.parameters["cut_track_HitAddChi2"]:
                # Save the hit
                hits_found.append(hits_thislayer[chi2_predict_inds[chi2_min_idx]])
                # Update the step and the Kalman filter
                step_pre = step_this
                mi = hits_found[-1]
                kf_find.forward_filter(np.array([mi.x, mi.z, mi.t]))

        return hits_found, kf_find.chift_total


    def find_in_layers_recursive(self, hits, hits_layer_grouped, layers_to_scan, kf_find, step_pre):
        """
        Find the hits that has minimum chi2 in total
        """
        self.found_hit_groups = []
        self.found_chi2_groups = []

        # Run the recrusive finding
        current_layer_ind = -1
        found_hits_inds = []
        found_chi2s = []
        self._find_in_layers_recursive(hits_layer_grouped, layers_to_scan, kf_find, step_pre, current_layer_ind, found_hits_inds, found_chi2s)

        if len(self.found_hit_groups)==0:
            return [], 0

        # Find the group with minimum chi2 per hit
        n_hits = [len(i) for i in self.found_hit_groups]
        chi2_reduced = np.sum(self.found_chi2_groups, axis=1)/n_hits
        ind_minchi2 = np.argmin(chi2_reduced)
        hits_found = [hits[i] for i in self.found_hit_groups[ind_minchi2] ]
        return hits_found, chi2_reduced
        
    def _find_in_layers_recursive(self, hits_layer_grouped, layers_to_scan, kf_find, step_pre, current_layer_ind, found_hits_inds, found_chi2s):
        current_layer_ind+=1 

        if current_layer_ind>=len(layers_to_scan):
            self.found_hit_groups.append(found_hits_inds)
            self.found_chi2_groups.append(found_chi2s)
            return

        layer = layers_to_scan[current_layer_ind]
        hits_thislayer = hits_layer_grouped[layer]

        # Get the prediction matrix 
        step_this = hits_thislayer[0].y 
        dy = step_this - step_pre # Step size
        step_pre = step_this
        _, Vi, Hi, Fi, Qi = KU.add_measurement(hits_thislayer[0], dy) # Calculate matrices. Only need to do once for all this in the same layer
        kf_find.update_matrix(Vi, Hi, Fi, Qi) # pass matrices to KF

        # Predicted location 
        Xp = kf_find.Xp_i
        Xp_unc = np.sqrt(np.diag(kf_find.Rp_i))
        # Function to test if new measurement is within N_sigma times the uncertainty ellipsoid
        N_sigma = self.parameters["cut_track_HitProjectionSigma"]
        test_measurement_compatible = lambda x,z,t: ((x-Xp[0])/Xp_unc[0])**2 + ((z-Xp[1])/Xp_unc[1])**2 + ((t-Xp[2])/Xp_unc[2])**2 <N_sigma

        # calculate chi2 for all hits in the next layer
        for imeasurement, m in enumerate(hits_thislayer):
            # Limit our search to the hits close to prediction:
            if not test_measurement_compatible(m.x, m.z, m.t):
                continue
            # print(m.x, m.z, m.t)
            # print(Xp[:3])
            # print(test_measurement_compatible(m.x, m.z, m.t))

            # Make copys for hits and chi2s for each recursion
            found_hits_inds_i = copy.deepcopy(found_hits_inds)
            found_chi2s_i = copy.deepcopy(found_chi2s)            
            kf_find_i = copy.deepcopy(kf_find)
            # Run Kalman filter
            chi2 = kf_find_i.forward_filter(np.array([m.x, m.z, m.t]))
            found_hits_inds_i.append(m.ind)
            found_chi2s_i.append(chi2)
            self._find_in_layers_recursive(hits_layer_grouped, layers_to_scan, kf_find_i, step_pre, current_layer_ind, found_hits_inds_i, found_chi2s_i)
                
        return

    def remove_related_hits_seeds(self, hits_found):
        hits_found_inds = [hit.ind for hit in hits_found]
        hits_found_inds.sort(reverse=True)
        # Remove seeds 
        # Need to do backwards to not change the index
        for i in reversed(range(len(self.seeds))):
            seed = self.seeds[i]
            if (seed[0] in hits_found_inds) or (seed[1] in hits_found_inds):
                self.seeds.pop(i)

        # Redo the grouping
        for layer in self.hits_grouped:
            hits = self.hits_grouped[layer]
            for ihit in reversed(range(len(hits))):
                if hits[ihit].ind in hits_found_inds:
                    hits.pop(ihit)


    def filter_smooth(self, hits, drop_chi2=-1):
        """
        Run the forward filter and backward smooth at once
        
        INPUT
        ---
        hits: list
            A list of all hits in a track
        drop_chi2: float
            for values less than zero, disable dropping
            for values zero, the steps with chi2 larger than this number will be dropped
        """
        kf = KF.KalmanFilter()

        # Set initial state using first two hits
        m0, V0, H0, Xf0, Cf0, Rf0 = KU.init_state(hits) # Use the first two hits to initiate
        kf.init_filter( m0, V0, H0, Xf0, Cf0, Rf0)
        

        # Feed all measurements to KF
        for i in range(2,len(hits)):   
            # get updated matrix
            hit = hits[i]
            dy  = hits[i].y-hits[i-1].y
            mi, Vi, Hi, Fi, Qi = KU.add_measurement(hit, dy)
            
            # pass to KF
            kf.forward_predict(mi, Vi, Hi, Fi, Qi)
            kf.forward_filter()

        # Filter backward
        dropped_inds = []
        if drop_chi2<0:
            kf.backward_smooth()
        else:
            # Manually go through all steps to check if it exceed drop_chi2
            kf.init_smooth()
            while kf.CURRENT_STEP>=0:
                chi2_temp = kf.smooth_step_try()

                dropped =  chi2_temp>drop_chi2
                if dropped:
                    dropped_inds.append(kf.CURRENT_STEP)
                # Finishing the current step
                kf.smooth_step(drop = dropped)


        

        return kf, dropped_inds 

    



In [6]:
# Test that it can find one track correctly

hits = Hit_collection[0]
hits_grouped = KU.group_hits_by_layer(hits)

tf = TF.TrackFinder(method="recursive")
# tf = TrackFinder(method="gredy")
seeds = tf.seeding(hits)
# seed=seeds[0]
seed=[0,10]
hits_found = tf.find_once(hits, hits_grouped, seed)
# [print(hit) for hit in hits_found]
hits_found


([Hit(x=30.19257514844793, y=1360, z=-126.0, t=6.978413091312879, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=2, ind=10)], 0)

In [21]:
reload(TF)

<module 'TrackFinder' from 'c:\\Users\\Tom\\OneDrive\\Documents\\JupyterNotebooks\\Physics\\TrackFinder.py'>

In [25]:
tf = TF.TrackFinder(method="recursive")
tf.find(hits)
print(tf.tracks_found)
tf.run(hits)
    

[[Hit(x=-72.94729582455193, y=1200, z=-27.0, t=3.9928815087908744, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=0, ind=12), Hit(x=-63.0, y=1280, z=-14.52133383783628, t=7.686967236969309, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=1, ind=13), Hit(x=-50.70722748615764, y=1360, z=-9.0, t=9.173465229742387, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=2, ind=14), Hit(x=-67.5, y=1440, z=-0.3071882254419269, t=10.924370250440711, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=3, ind=15)], [Hit(x=15.850727352347342, y=1200, z=-103.5, t=1.5012461903677357, x_err=14.132352000025548, y_err=0, z_err=1.299038105676658, t_err=1, layer=0, ind=8), Hit(x=18.0, y=1280, z=-111.2970066475968, t=4.769708687793716, x_err=1.299038105676658, y_err=0, z_err=14.132352000025548, t_err=1, layer=1, ind=9), Hit(x=30.19257514844793, y=1360, z=-126.0, t=6.978413091312879, x_err=14.132352000025548,

[Track(x0=-60.942191217439984, y0=1200, z0=-26.98402880500378, t=4.602276423827433, Ax=-0.026857082923317066, Ay=1, Az=0.11246671330925626, At=0.02785120527215323, cov=array([[ 4.12807664e+00,  0.00000000e+00,  0.00000000e+00,
         -2.06577639e-02,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.68043024e+00,  0.00000000e+00,
          0.00000000e+00, -1.04592372e-02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  7.00000000e-01,
          0.00000000e+00,  0.00000000e+00, -3.75000000e-03],
        [-2.06577639e-02,  0.00000000e+00,  0.00000000e+00,
          1.29654171e-04,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.04592372e-02,  0.00000000e+00,
          0.00000000e+00,  1.29654171e-04,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -3.75000000e-03,
          0.00000000e+00,  0.00000000e+00,  3.12500000e-05]]), chi2=3.101679664835769, ind=0, hits=[12, 13, 14, 15], hits_filtered=None),
 Track(x0=18.005808575216616, 

In [26]:
Truth_collection[0]

[Hit(x=97.68637490357727, y=1200, z=31.704928526359936, t=-1.7345464325364786, x_err=0, y_err=0, z_err=0, t_err=0, layer=0, ind=0),
 Hit(x=95.60557933748451, y=1280, z=15.439160046297864, t=0.795267167840668, x_err=0, y_err=0, z_err=0, t_err=0, layer=1, ind=1),
 Hit(x=93.52478377139175, y=1360, z=-0.826608433764207, t=3.3250807682178145, x_err=0, y_err=0, z_err=0, t_err=0, layer=2, ind=2),
 Hit(x=91.44398820529901, y=1440, z=-17.09237691382628, t=5.8548943685949615, x_err=0, y_err=0, z_err=0, t_err=0, layer=3, ind=3),
 Hit(x=1.2580760709354166, y=1200, z=3.216656074541973, t=-0.1957374407239456, x_err=0, y_err=0, z_err=0, t_err=0, layer=0, ind=0),
 Hit(x=16.83083528333231, y=1280, z=12.890914290149487, t=2.3574752880972176, x_err=0, y_err=0, z_err=0, t_err=0, layer=1, ind=1),
 Hit(x=32.4035944957292, y=1360, z=22.565172505757, t=4.910688016918381, x_err=0, y_err=0, z_err=0, t_err=0, layer=2, ind=2),
 Hit(x=47.97635370812609, y=1440, z=32.23943072136451, t=7.463900745739544, x_err=0, y_

## Run on an event that failed in the tracker

In [52]:
x = [241.62934581783023, 259.25, 221.49664808213277, 227.75]
y = [9894.0, 9975.599999999999, 10057.2, 10138.8]
z = [12456.75, 12450.5, 12461.25, 12450.5]
t = [40.91365706617539, 45.79668867914941, 48.189471653643565, 50.295569678622876]

hits = debug.make_hits(x,y,z,t)

tf = TF.TrackFinder(method="recursive")
# tf.find(hits)
# print(tf.tracks_found)
tf.run(hits)

[Track(x0=274.2196119526758, y0=9894.0, z0=12456.768104297438, t=41.71806864762234, Ax=-0.18964018097796365, Ay=1, Az=0.026258905960041642, At=0.0374246578576429, cov=array([[ 4.12807664e+00,  0.00000000e+00,  0.00000000e+00,
         -2.02527097e-02,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.68043024e+00,  0.00000000e+00,
          0.00000000e+00, -1.02541541e-02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  7.00000000e-01,
          0.00000000e+00,  0.00000000e+00, -3.67647059e-03],
        [-2.02527097e-02,  0.00000000e+00,  0.00000000e+00,
          1.24619542e-04,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.02541541e-02,  0.00000000e+00,
          0.00000000e+00,  1.24619542e-04,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -3.67647059e-03,
          0.00000000e+00,  0.00000000e+00,  3.00365244e-05]]), chi2=11.199094002959015, ind=0, hits=[0, 1, 2, 3])]